# How to submit to Quantinuum H-series Devices?


In [3]:
import pytket
import pytket.extensions.quantinuum


print("pytket version: ", pytket.__version__)
print("pytket-extensions-quantinuum version: ", pytket.extensions.quantinuum.__extension_version__)

pytket version:  1.25.0
pytket-extensions-quantinuum version:  0.30.0


## Circuit Preparation

In [4]:
from pytket.circuit import Circuit, fresh_symbol
from pytket.circuit.display import render_circuit_jupyter


# set up a bell test
circuit = Circuit(2, name="Bell State Test")
circuit.H(0) # Hadamard gate
circuit.CX(0, 1) # CNOT gate
circuit.measure_all() # measure all qubits

# render the circuit
render_circuit_jupyter(circuit)

## Select Device

In [5]:
from pytket.extensions.quantinuum import QuantinuumBackend

machine = 'H1-1E'

backend = QuantinuumBackend(device_name= machine)
backend.login()

Get the status of the machine

In [6]:
print(machine, "status: ", QuantinuumBackend.device_state(device_name=machine))

H1-1E status:  online


## Circuit Compilation

In [7]:
compiled_circuit = backend.get_compiled_circuit(circuit,optimisation_level=2)
render_circuit_jupyter(compiled_circuit)

## Circuit Cost

In [9]:
n_shots = 100
backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker='H1-1SC')

5.66

## Run the Circuit

In [10]:
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)
print(handle)

('34e422fb08144f23a36d4bc535906d89', 'null', 2, '[["c", 0], ["c", 1]]')


In [11]:
import json 
with open('pytket_example_job_handle.json','w') as file:
    json.dump(str(handle), file)

## Check Job Status

In [12]:
status = backend.circuit_status(handle)
print(status)

CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Bell State Test", "submit-date": "2024-03-08T09:12:42.659612", "result-date": "2024-03-08T09:12:51.475581", "queue-position": null, "cost": "5.66", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


## Retrieve Results

In [13]:
from pytket.backends import ResultHandle

with open('pytket_example_job_handle.json', 'r') as file:
    handle_str = json.load(file)

handle = ResultHandle.from_str(handle_str)

In [14]:
result = backend.get_result(handle)

## Save Results

In [16]:
import json 
with open('pytket_example.json','w') as file:
    json.dump(result.to_dict(), file)

In [17]:
from pytket.backends.backendresult import BackendResult
import json
with open('pytket_example.json') as file:
    data = json.load(file)

result = BackendResult.from_dict(data)

## Analyze Results

In [19]:
print(result.get_counts())
print("-----------------------")
print(result.get_distribution())

Counter({(0, 0): 57, (1, 1): 42, (1, 0): 1})
-----------------------
{(0, 0): 0.57, (1, 0): 0.01, (1, 1): 0.42}


# Additional Features

## Batch Submission

In [28]:

machine = 'H1-1E'

backend = QuantinuumBackend(device_name= machine)

print(machine, "status: ", QuantinuumBackend.device_state(device_name=machine))

H1-1E status:  online


In [29]:
machine = 'H1-1E'
n_shots = 1000

max_batch_cost = 100

backend = QuantinuumBackend(device_name=machine)

In [30]:


batch1 = backend.start_batch(max_batch_cost=max_batch_cost, circuit= compiled_circuit, n_shots=n_shots)

Additional Jobs can be added to the batch using the `add_to_batch` function. The end of a batch can optionally be specified with `batch_end` flag

In [31]:
batch2 = backend.add_to_batch(batch1, compiled_circuit, n_shots=n_shots)
batch3 = backend.add_to_batch(batch1, compiled_circuit, n_shots=n_shots, batch_end=True)

The status for the batch jobs can be checked once submitted

In [34]:
handle_list = [batch1, batch2, batch3]

status_list = [backend.circuit_status(handle) for handle in handle_list]

status_list

[CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Bell State Test", "submit-date": "2024-03-08T09:30:01.154755", "result-date": "2024-03-08T09:30:24.109786", "queue-position": null, "cost": "11.6", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None),
 CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Bell State Test", "submit-date": "2024-03-08T09:30:02.991084", "result-date": "2024-03-08T09:30:20.257963", "queue-position": null, "cost": "11.6", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None),
 CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Bell State Test", "submi

Results for the batch submissions can be returned using `get_results`

In [35]:
result = backend.get_results(handle_list)

result

[BackendResult(q_bits={},c_bits={c[0]: 0, c[1]: 1},counts=None,shots=[[192]
  [  0]
  [192]
  [192]
  [  0]
  [192]
  [192]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [192]
  [192]
  [192]
  [192]
  [  0]
  [192]
  [192]
  [192]
  [192]
  [  0]
  [  0]
  [192]
  [192]
  [  0]
  [  0]
  [192]
  [ 64]
  [192]
  [192]
  [192]
  [192]
  [  0]
  [192]
  [  0]
  [  0]
  [  0]
  [192]
  [  0]
  [192]
  [192]
  [192]
  [192]
  [192]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [192]
  [  0]
  [192]
  [  0]
  [  0]
  [  0]
  [  0]
  [192]
  [192]
  [  0]
  [192]
  [192]
  [192]
  [192]
  [  0]
  [  0]
  [  0]
  [192]
  [192]
  [  0]
  [  0]
  [192]
  [192]
  [192]
  [  0]
  [192]
  [192]
  [  0]
  [192]
  [  0]
  [  0]
  [  0]
  [  0]
  [192]
  [  0]
  [  0]
  [  0]
  [192]
  [192]
  [192]
  [192]
  [192]
  [192]
  [  0]
  [192]
  [192]
  [  0]
  [  0]
  [  0]
  [192]
  [  0]
  [192]
  [  0]
  [192]
  [192]
  [192]
  [192]
  [  0]
  [  0]
  [  0]
  [  0]
  [192]
  [  0]
  [  0]
  [  0]
  [ 

## Parameterized Circuits

In [36]:
from pytket.circuit import fresh_symbol

# set up a parameterised circuit
a = fresh_symbol('a')
circuit = Circuit(3, name="Parameterised Circuit")
circuit.X(0) # Pauli-X gate
circuit.CX(0,1).CX(1,2) # CNOT gate 

circuit.Rz(a, 2) # Rotation-Z gate
circuit.CX(1,2).CX(0,1) # CNOT gate

render_circuit_jupyter(circuit)

Now create a similar circuit, but with values of `a`

In [ ]:
simulation_circuit = circuit.copy()